In [1]:
"""

README 
NEED TO CHANGE THE PATHS TO THE CHECK IMAGES

1:dir_to_process 
2: output_dir_for_extracted_data



dir_to_process must be made and contain the jpg images of the checks to be processed

output_dir_for_extracted_data will be created and contain the unstructured gpt4v data in a text file and the gpt4 extractor tool formated data in json format


AND Update OpenAI key in the openai.api_key = os.environ['WATCH_TOWER_OPENAI_KEY'] line

openai_api_key = os.environ['WATCH_TOWER_OPENAI_KEY']
openai.api_key = os.environ['WATCH_TOWER_OPENAI_KEY']


"""

import openai
import os

#Base path for the check image
#TEST ONE: NO MEMO FIELD
dir_to_process = "app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65caaffb9b6ff316a779f525/check_images_meta/"
output_dir_for_extracted_data = "app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65caaffb9b6ff316a779f525/extracted_data/"
#TODO: once done apply to above
#dir_to_process = "app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65caaffb9b6ff316a779f525/small test/"



#TEST Two: WITH MEMO FIELD
dir_to_process ="app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/"
output_dir_for_extracted_data = "app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/extracted_data/"
#Example of the fields in the check
#TODO: UPDATE
#check_visual_prompting_example_path = "/app/FinAware/visual prompting of a check.jpg"
#base64_check_visual_prompting_example_image = encode_image(check_visual_prompting_example_path)

#NOTE: In production turn this off.
include_images_with_results = False

#More space because saves images with the results but easier to debug and review the results. Can click on both results and image
include_images_with_results = True

In [2]:
#Altered version of finaware_check_dat_extract.ipynb
#Supports dir input of jpg files

#UPDATE THE PATHS TO YOUR OPENAI API KEY
openai_api_key = os.environ['WATCH_TOWER_OPENAI_KEY']
openai.api_key = os.environ['WATCH_TOWER_OPENAI_KEY']

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

#Personal API key
#openai.api_key = os.environ['OPENAI_API_KEY']

#Watch Tower API Key


import base64
import requests

In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from typing import List
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.messages import HumanMessage, AIMessage

In [4]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')



In [5]:
system_message = """
You are a bank teller's tool to assist in manual data entry of checks. You are designed to extract and validate information from personal or business checks, including the check creation date, check maker's full name and address, check maker's phone number, check maker's bank information, payee information, signature line, check amount (numerical and written), check number, routing number, and account number.
You are also responsible for performing necessary pre-processing steps on the input image to enhance the readability of the text, such as image normalization, denoising, and binarization. You must ensure that the extracted data is accurate and correctly formatted, correcting any OCR misreads or errors.
Your goal is to provide the bank teller with structured information that can be easily verified and processed.

Input: Accept an image of a personal or business check.
Image Pre-processing: Perform necessary pre-processing steps on the input image to enhance the readability of the text. This may include image normalization, denoising, and binarization to prepare the image for OCR.
OCR Execution: Apply OCR technology to the pre-processed image to extract textual content from the entire check image.

Text Parsing and Extraction:
Check Creation Date: Identify and extract the date the check is written on. Validate that this date is not in the future.
Check Maker's Full Name and Address: Locate the top left corner of the check for the check maker's full name and address. Ensure the entire address is captured, including city, state, and zip code.
Check Maker's Phone Number: If present, extract the phone number of the person or business who wrote the check, including the area code.
Check Maker's Bank Information: Extract the bank name and address from the check, focusing on any visible details that indicate the financial institution.
Payee Information: Identify the "Pay to the order of" field to extract the name of the person or business the check is written to.
Signature Line: Locate and confirm the presence of a signature on the signature line.
Check Amount (Numerical): Extract the numerical amount of the check, typically found on the right side.
Check Amount (Words): Extract the check amount written in words, usually located on the line below the payee's name.
Bottom Check Number: Identify and extract the check number from the MICR line at the bottom of the check.
Top Right Check Number: Extract the check number located in the top right corner of the check.
Routing Number: Extract the 9-digit ABA routing number from the MICR format on the bottom left of the check.
Account Number: Identify and extract the 10-12 digit account number from the MICR line on the bottom left of the check.
Data Validation and Correction:
Perform validation checks on extracted data to ensure accuracy. This includes verifying the format of dates, phone numbers, routing numbers, and account numbers.
Correct any OCR misreads or errors in the extracted text based on common patterns or validation rules.
Output: Structure the extracted information according to the specified schema, making each piece of information available as a separate feature for further processing or verification by bank tellers.
"""

In [6]:
model_version = "gpt-4-vision-preview" 
#model_version ="gpt-4-1106-vision-preview"
model = ChatOpenAI(temperature=0, model = model_version )

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
from typing import Optional
"""
Checks and balances 

Could be differences in values 
check_amount == check_word_amount

TODO: bottom_check_number can have a leading 0 which is not seen in the top_right_check_number. 
TODO: Remove the leading 0 from bottom_check_number and compare with top_right_check_number 
Bottom could be written from the signature line.
bottom_check_number == top_right_check_number

Check maker is the person/business who wrote the check
"""

class BankTransaction(BaseModel):
    """A bank transaction statement entry."""
    posting_date: str = Field(description="The date the transaction was posted.")
    transaction_description: str = Field(description="Description of the transaction.")
    deposits_credits: Optional[float] = Field(description="Amount of deposits or credits in the transaction.Determines the amount of money that is being added to the account.Consider if numbers are negative or positive.")
    withdrawals_debits: Optional[float] = Field(description="Amount of withdrawals or debits in the transaction.Determines the amount of money that is being added to the account.Consider if numbers are negative or positive.")
    daily_balance: float = Field(description="The daily balance after the transaction.")




#Same name on signup form
class bank_check(BaseModel):
    """A bank or personal check."""
    check_creation_date: str = Field(description="The date the check is written (cannot be a future date)")
    check_maker_full_name: str = Field(description="Full name of the Person/business who wrote the check usually in the top left corner on the check.")
    check_maker_address: str = Field(description="Address of the Person/business who wrote the check usually in the top left corner on the check. Capture the whole address including city, state, and zip code.")
    check_maker_phone_number: Optional[str] = Field(description="Phone number of the Person/business who wrote the check usually in the top left corner on the check. Capture the whole phone number including area code.")
    check_maker_bank_name: Optional[str] = Field(description="Bank name of the Person/business who wrote the check.")
    check_maker_bank_address: Optional[str] = Field(description="Bank address of the Person/business who wrote the check.")
    check_maker_bank_info: Optional[str] = Field(description="Any additional Bank information of the Person/business who wrote the check.")
    payee: str = Field(description="Person/business to whom the check is written. Usually next to the word 'Pay to the order of' on the check.")
    signature_line: str= Field(description="Signature line on the check. Describe anything remarkable about the signature line.")
    check_amount: str= Field(description="The amount of the check written in numbers, usually on the right side of the check")
    check_word_amount: str= Field(description="The amount of the check written in words, usually on the line below the payee's name")
    bottom_check_number: int= Field(description="Printed on the check and appears in the MICR line on the bottom of the check.")
    top_right_check_number: int= Field(description="Printed on the check and appears in the top right corner of the check.")
    routing_number : int= Field(description="ABA Routing Number (MICR Format). The first set of numbers on the bottom left of the check. It is a 9-digit number")
    account_number : int= Field(description="The second set of numbers in MICR Format on the bottom left of the check. It is a 10-12 digit number")
    check_memo_or_for : str = Field(description="Memo or For on the check. Can be used to describe the purpose of the check or any other relevant information.")
    ocr_errors: Optional[str] = Field(description="Describe any fields of the check that might be questionable or erroneous due to OCR or logical errors.")

    
    
class money_order(BaseModel):
    """A money order."""
    payee_name : Optional[str] = Field(description="The payee's name i.e. the person or company the money order is being made out to")
    payee_address: str = Field(description="The payee's address")
    money_order_memo : str = Field(description="The memo on the money order")
    purchaser_name : Optional[str] = Field(description="The purchaser's name on the money order")
    purchaser_address : Optional[str] = Field(description="The purchaser's address on the money order")
    amount: float = Field(description="The amount of the money order")

#payee_account_number: Optional[str] = Field(description="The payee's bank account number where the money order is being deposited")

In [8]:
extractor_model_non_vision = ChatOpenAI(temperature=0)

extraction_functions = [convert_pydantic_to_openai_function(bank_check)]
#CANT use tool usage on gpt4V currently as of 2-14-2024
extraction_model = extractor_model_non_vision.bind(functions=extraction_functions, function_call={"name": "bank_check"})

/home/vscode/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [9]:

#This would be applied to the output from gpt4v model which will be raw and verbose, this model will summarize.
prompt_wo_image = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("human", "{input}")
    
    
])

In [10]:
extraction_chain = prompt_wo_image | extraction_model | JsonOutputFunctionsParser()

In [11]:

#openai_api_key = os.getenv('OPENAI_API_KEY')

#print(openai_api_key)
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {openai_api_key}"
}


In [13]:
import json
import glob
import shutil
# Get a list of all jpg files in the directory
jpg_files = glob.glob(f"{dir_to_process}/*.jpg")

#print(jpg_files)


for jpg_file in jpg_files:
    # Get the full path to the image
    full_path = os.path.abspath(jpg_file)
    print(full_path)
    
    image_name = os.path.basename(full_path)
    path_without_file = os.path.dirname(full_path)
    image_name_without_extension = os.path.splitext(image_name)[0]
    
    #Analytics file to see if the file was processed
    file_path = os.path.join(output_dir_for_extracted_data, f"{image_name_without_extension}_unstructured.txt")
    
    
    

    # Check if the output directory exists
    if not os.path.exists(output_dir_for_extracted_data):
      # If not, create the directory
      os.makedirs(output_dir_for_extracted_data)
    
    if not os.path.exists(file_path):
      
      if include_images_with_results:
        #Move images into folder with results for easier viewing. 
        # Construct the destination path
        copy_of_check_destination_path = os.path.join(output_dir_for_extracted_data, image_name)

        # Copy the image
        #shutil.copy(full_path, output_dir_for_extracted_data)
        shutil.copy(full_path, copy_of_check_destination_path)
    
    
      # Pass the full path to the encode_image function
      base64_check_image=encode_image(full_path)




      """
      CAN'T use tool usage in gpt4v so going to use this method to get the output
      """

      text_input_prompt_second = "Please help me in data entry of this check."


      payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
          {"role": "system", "content": system_message},
          
            {"role": "user",
            "content": [
              {
                "type": "text",
                "text": text_input_prompt_second
              },
              {
                "type": "image_url",
                "image_url": {
                  "url": f"data:image/jpeg;base64,{base64_check_image}","detail": "high"
                }
              }]
          }
            
              
          ], "max_tokens": 1000
          }
      
      
      response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
      
      try:
        unpacked=response.json()
      except Exception as e:
        print(f"Error: {full_path}")
        with open(f'{output_dir_for_extracted_data}/unpack_error.txt', 'a') as f:
          f.write(f"Error:{e} {full_path}\n")
    
      unstructured_content=unpacked['choices'][0]['message']['content']
      #print(unstructured_content)
      
      
      
      
      #Saving the first pass of the gpt4v model without tools. Text is unstructured. Output from this model is used as input into another model
      with open(f"{output_dir_for_extracted_data}/{image_name_without_extension}_unstructured.txt", 'w') as f:
          f.write(unstructured_content)

      
      try:
        response_formatted = extraction_chain.invoke({"input":unstructured_content})
        
      except json.JSONDecodeError as e:
          print(f"JSONDecodeError: {e}{full_path}")
          with open(f'{output_dir_for_extracted_data}/errored_files.txt', 'a') as f:
            f.write(f"JSONDecodeError: {e} {full_path}\n")
            
      except Exception as e:
          print(f"Error: {full_path}")
          with open(f'{output_dir_for_extracted_data}/errored_files.txt', 'a') as f:
            f.write(f"Error:{e} {full_path}\n")
      
      
      json_data = json.dumps(response_formatted)
      #Saving the second pass of the gpt4 model with structured output tools. Text is structured to meet the bank_check model
      #with open(f"{path_without_file}/{image_name_without_extension}_extracted.json", 'w') as f:
      with open(f"{output_dir_for_extracted_data}/{image_name_without_extension}_extracted.json", 'w') as f:
          f.write(json_data)
    
    
    else: 
        print(f"File exists not computing again: {file_path}")



/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_38_0_0.jpg


/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_38_0_1.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_38_0_2.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_46_0_0.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_46_0_1.jpg
Error: /app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_46_0_1.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_46_0_2.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/65a8168eb3ac164610ea5bc2/check_images_meta/NEW AGE VENDING LLC_check_46_0_3.jpg
/app/FinAware/CHECK_MEDIA_STORAGE/CHECK_MEDIA_STORAGE/6

 except OutputParserException as e:
          print(f"OutputParserException: {e}{full_path}")
          with open('{output_dir_for_extracted_data}/errored_files.txt', 'a') as f:
            f.write(f"OutputParserException: {e} {full_path}\n")

In [ ]:

"""

output from gpt4v without tool usage need to transform into json, so will need one more extractor LLM ( that's not vision) 
to transform into json because gpt4V can't use tools
"""